# Tool Calling: Building Agentic Workflows

This notebook demonstrates how to build agentic workflows using lionpride's **ReAct** (Reasoning + Acting) pattern. You'll learn how to:

- Define tools that LLMs can use
- Use `react()` for multi-step reasoning loops
- Build practical agents with real-world tools
- Handle errors and structured outputs

**What you'll build**: Calculator assistant, research agent, data analyst, and weather advisor.

## What is Tool Calling?

**Tool calling** enables LLMs to interact with external functions, APIs, and systems. Instead of just generating text, the LLM can:

1. **Reason** about what action to take
2. **Act** by calling a tool with arguments
3. **Observe** the tool's result
4. **Repeat** until it has enough information to answer

This is the **ReAct pattern** (Reason + Act):

```
Instruction → Reasoning → Tool Call → Observation → Reasoning → ... → Final Answer
```

**lionpride's `react()`** operation implements this loop with:
- Automatic tool schema generation
- Multi-step reasoning control (`max_steps`)
- Structured output support
- Error handling and retries

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install lionpride pydantic python-dotenv

In [ ]:
from datetime import datetime

# Load API keys from environment
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from lionpride import Session
from lionpride.operations import react
from lionpride.services import Tool, iModel

load_dotenv()

print("✅ Imports successful")

## Example 1: Calculator Assistant

Let's build a simple math assistant that can add and multiply numbers. The LLM will break down complex calculations into tool calls.

In [ ]:
# Define calculator tools
# Note: Tool names must be at least 4 characters
def add_nums(a: float, b: float) -> float:
    """Add two numbers together"""
    result = a + b
    print(f"  add_nums({a}, {b}) = {result}")
    return result


def multiply_nums(a: float, b: float) -> float:
    """Multiply two numbers together"""
    result = a * b
    print(f"  multiply_nums({a}, {b}) = {result}")
    return result


def subtract_nums(a: float, b: float) -> float:
    """Subtract b from a"""
    result = a - b
    print(f"  subtract_nums({a}, {b}) = {result}")
    return result


def divide_nums(a: float, b: float) -> float:
    """Divide a by b"""
    if b == 0:
        raise ValueError("Cannot divide by zero!")
    result = a / b
    print(f"  divide_nums({a}, {b}) = {result}")
    return result

In [ ]:
async def calculator_agent():
    """Math assistant using calculator tools"""
    # Create session and model
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)

    # Create tools from functions using Tool(func_callable=)
    add_tool = Tool(func_callable=add_nums)
    multiply_tool = Tool(func_callable=multiply_nums)
    subtract_tool = Tool(func_callable=subtract_nums)
    divide_tool = Tool(func_callable=divide_nums)

    # Note: Tools are automatically registered by react() - no need to register manually

    # Create branch for conversation
    branch = session.create_branch(name="calculator")

    # Run ReAct loop
    print("Question: What is (15 + 27) * 3 - 10?\n")

    result = await react(
        session=session,
        branch=branch,
        parameters={
            "instruction": "What is (15 + 27) * 3 - 10?",
            "imodel": model.name,
            "model_name": model.name,  # Required for tool schema generation
            "tools": [add_tool, multiply_tool, subtract_tool, divide_tool],  # Pass Tool instances
            "max_steps": 5,
        },
    )

    print(f"\nFinal Answer: {result.final_response}")
    print(f"Steps taken: {result.total_steps}")
    print(f"Completed: {result.completed}")

    return result


# Run the calculator agent
result = await calculator_agent()

### Inspecting the ReAct Trace

Let's examine how the LLM reasoned through the problem step-by-step.

In [ ]:
print("\n📋 Detailed Reasoning Trace:\n")
print("=" * 80)

for step in result.steps:
    print(f"\n🔄 Step {step.step}:")
    print(f"   Reasoning: {step.reasoning[:150]}...")  # Truncate for readability

    if step.actions_requested:
        for action in step.actions_requested:
            print(f"   🛠️  Tool: {action.function}")
            print(f"   📥 Args: {action.arguments}")

    if step.actions_executed:
        for action in step.actions_executed:
            print(f"   📤 Result: {action.output}")

print("\n" + "=" * 80)

## Example 2: Research Agent with Web Search

Build an agent that can search the web and retrieve current information. We'll simulate a search API.

In [ ]:
# Define research tools
def search_web(query: str, num_results: int = 3) -> list[dict]:
    """
    Search the web for information.

    Args:
        query: Search query string
        num_results: Number of results to return (1-5)

    Returns:
        List of search results with title, snippet, and url
    """
    print(f"  🔍 Searching web for: '{query}'")

    # Simulated search results based on query
    if "AI" in query or "artificial intelligence" in query.lower():
        results = [
            {
                "title": "OpenAI Announces GPT-5 Development",
                "snippet": "OpenAI confirms work on next-generation language model with enhanced reasoning capabilities...",
                "url": "https://openai.com/news/gpt5",
            },
            {
                "title": "DeepMind's AlphaFold 3 Breakthrough",
                "snippet": "New protein folding predictions achieve unprecedented accuracy for drug discovery...",
                "url": "https://deepmind.com/alphafold3",
            },
            {
                "title": "EU AI Act Becomes Law",
                "snippet": "European Union implements comprehensive AI regulation framework...",
                "url": "https://ec.europa.eu/ai-act",
            },
        ]
    elif "Python" in query:
        results = [
            {
                "title": "Python 3.13 Released",
                "snippet": "Latest Python version features performance improvements and new syntax...",
                "url": "https://python.org/3.13",
            },
            {
                "title": "FastAPI 0.110 Update",
                "snippet": "Modern web framework adds native async support...",
                "url": "https://fastapi.tiangolo.com",
            },
        ]
    else:
        results = [
            {
                "title": f"Result {i + 1} for '{query}'",
                "snippet": f"This is information about {query}...",
                "url": f"https://example.com/result{i + 1}",
            }
            for i in range(num_results)
        ]

    return results[:num_results]


def get_current_date() -> str:
    """Get the current date in YYYY-MM-DD format"""
    date = datetime.now().strftime("%Y-%m-%d")
    print(f"  📅 Current date: {date}")
    return date


def get_current_time() -> str:
    """Get the current time in HH:MM:SS format"""
    time = datetime.now().strftime("%H:%M:%S")
    print(f"  🕐 Current time: {time}")
    return time

In [ ]:
async def research_agent():
    """Research assistant using search tools"""
    session = Session()
    model = iModel(
        provider="anthropic", endpoint="messages", model="claude-3-5-sonnet-20241022", temperature=0
    )
    session.services.register(model)

    # Create tools using Tool(func_callable=)
    search_tool = Tool(func_callable=search_web)
    date_tool = Tool(func_callable=get_current_date)
    time_tool = Tool(func_callable=get_current_time)

    # Note: Tools are automatically registered by react() - no need to register manually

    branch = session.create_branch(name="research")

    print("🤔 Question: What are the major AI breakthroughs this year?\n")

    result = await react(
        session=session,
        branch=branch,
        parameters={
            "instruction": "What are the major AI breakthroughs this year? Give me 3 key developments.",
            "imodel": model.name,
            "model_name": model.name,  # Required for tool schema generation
            "tools": [search_tool, date_tool, time_tool],  # Pass Tool instances
            "max_steps": 5,
        },
    )

    print(f"\n📝 Research Result:\n{result.final_response}")

    return result


# Run research agent
result = await research_agent()

## Example 3: Data Analyst Agent

Build an agent that queries a database and performs calculations. This demonstrates multi-step reasoning with data.

In [ ]:
# Simulated employee database
DATABASE = {
    "employees": [
        {"id": 1, "name": "Alice Chen", "department": "Engineering", "salary": 120000, "years": 5},
        {"id": 2, "name": "Bob Smith", "department": "Sales", "salary": 90000, "years": 3},
        {"id": 3, "name": "Charlie Kim", "department": "Engineering", "salary": 110000, "years": 4},
        {"id": 4, "name": "Diana Lopez", "department": "Sales", "salary": 95000, "years": 2},
        {"id": 5, "name": "Eve Johnson", "department": "Engineering", "salary": 130000, "years": 7},
        {"id": 6, "name": "Frank Wilson", "department": "Marketing", "salary": 85000, "years": 3},
    ],
    "projects": [
        {"id": 1, "name": "AI Platform", "budget": 500000, "team": "Engineering"},
        {"id": 2, "name": "Sales Campaign", "budget": 200000, "team": "Sales"},
    ],
}


def query_database(table: str, filter_by: str | None = None) -> list[dict]:
    """
    Query a database table with optional filtering.

    Args:
        table: Table name (e.g., 'employees', 'projects')
        filter_by: Optional filter like 'department=Engineering' or 'years>3'

    Returns:
        List of matching records
    """
    print(f"  💾 Querying table: {table} (filter: {filter_by or 'none'})")

    data = DATABASE.get(table, [])

    if filter_by:
        # Parse simple filters like "key=value" or "key>value"
        if "=" in filter_by:
            key, value = filter_by.split("=")
            data = [row for row in data if str(row.get(key)) == value]
        elif ">" in filter_by:
            key, value = filter_by.split(">")
            data = [row for row in data if row.get(key, 0) > int(value)]
        elif "<" in filter_by:
            key, value = filter_by.split("<")
            data = [row for row in data if row.get(key, 0) < int(value)]

    print(f"  📊 Found {len(data)} records")
    return data


def calculate_average(numbers: list[float]) -> float:
    """Calculate the average of a list of numbers"""
    if not numbers:
        return 0.0
    avg = sum(numbers) / len(numbers)
    print(f"  📈 Average of {len(numbers)} values: {avg:,.2f}")
    return avg


def calculate_sum(numbers: list[float]) -> float:
    """Calculate the sum of a list of numbers"""
    total = sum(numbers)
    print(f"  ➕ Sum of {len(numbers)} values: {total:,.2f}")
    return total

In [ ]:
async def data_analyst_agent():
    """Data analyst using database query tools"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o", temperature=0)
    session.services.register(model)

    # Create data analysis tools using Tool(func_callable=)
    query_tool = Tool(func_callable=query_database)
    avg_tool = Tool(func_callable=calculate_average)
    sum_tool = Tool(func_callable=calculate_sum)

    # Note: Tools are automatically registered by react() - no need to register manually

    branch = session.create_branch(name="analysis")

    print(
        "🤔 Question: What is the average salary of employees in Engineering with more than 4 years of experience?\n"
    )

    result = await react(
        session=session,
        branch=branch,
        parameters={
            "instruction": (
                "What is the average salary of employees in the Engineering department "
                "who have more than 4 years of experience?"
            ),
            "imodel": model.name,
            "model_name": model.name,  # Required for tool schema generation
            "tools": [query_tool, avg_tool, sum_tool],  # Pass Tool instances
            "max_steps": 5,
        },
    )

    print(f"\n💼 Analysis Result: {result.final_response}")

    # Show tool execution trace
    print("\n🔍 Tool Execution Trace:")
    for step in result.steps:
        if step.actions_executed:
            for action in step.actions_executed:
                print(f"   {action.function}({action.arguments}) → {action.output}")

    return result


# Run data analyst agent
result = await data_analyst_agent()

## Example 4: Error Handling in Tools

See how the ReAct loop handles tool errors gracefully. The LLM observes the error and can retry with corrected arguments.

In [ ]:
# Note: Tool names must be at least 4 characters
def safe_divide(a: float, b: float) -> float:
    """Divide a by b with error handling"""
    if b == 0:
        print("  Error: Cannot divide by zero!")
        raise ValueError("Division by zero is not allowed")
    result = a / b
    print(f"  safe_divide({a}, {b}) = {result}")
    return result


def get_number_info(n: int) -> dict:
    """Get information about a number"""
    if n < 0:
        print("  Error: Negative numbers not supported!")
        raise ValueError("Number must be non-negative")

    info = {
        "number": n,
        "is_even": n % 2 == 0,
        "is_prime": n > 1 and all(n % i != 0 for i in range(2, int(n**0.5) + 1)),
        "square": n**2,
    }
    print(f"  Info for {n}: even={info['is_even']}, prime={info['is_prime']}")
    return info

In [ ]:
async def error_handling_demo():
    """Demonstrate error handling in ReAct loop"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)

    # Create tools using Tool(func_callable=)
    divide_tool = Tool(func_callable=safe_divide)
    info_tool = Tool(func_callable=get_number_info)

    # Note: Tools are automatically registered by react() - no need to register manually

    branch = session.create_branch(name="error_demo")

    print("🤔 Question: Divide 100 by (5 - 5) and tell me about the result\n")
    print("(This will trigger a division by zero error, then recover)\n")

    result = await react(
        session=session,
        branch=branch,
        parameters={
            "instruction": (
                "First divide 100 by (5 - 5), then give me information about the result."
            ),
            "imodel": model.name,
            "model_name": model.name,  # Required for tool schema generation
            "tools": [divide_tool, info_tool],  # Pass Tool instances
            "max_steps": 5,
        },
    )

    print(f"\n🎯 Result: {result.final_response}")
    print("\n📝 Note: The LLM observed the error and corrected its approach")

    return result


# Run error handling demo
result = await error_handling_demo()

## Example 5: Structured Output with Tool Calling

Combine tool calling with structured outputs using Pydantic models. The LLM gathers information via tools, then returns a typed response.

In [ ]:
# Define structured output model
class WeatherReport(BaseModel):
    """Complete weather report with recommendations"""

    city: str = Field(..., description="City name")
    temperature: float = Field(..., description="Temperature in Fahrenheit")
    condition: str = Field(..., description="Weather condition (e.g., Sunny, Rainy)")
    humidity: int = Field(..., description="Humidity percentage")
    recommendation: str = Field(..., description="What to wear and bring")


def get_weather(city: str) -> dict:
    """Get current weather for a city"""
    print(f"  🌤️  Fetching weather for: {city}")

    # Simulated weather API
    weather_data = {
        "San Francisco": {"temp": 65, "condition": "Foggy", "humidity": 75},
        "New York": {"temp": 78, "condition": "Sunny", "humidity": 60},
        "Seattle": {"temp": 55, "condition": "Rainy", "humidity": 85},
        "Miami": {"temp": 88, "condition": "Humid", "humidity": 80},
        "Chicago": {"temp": 70, "condition": "Cloudy", "humidity": 65},
    }

    data = weather_data.get(city, {"temp": 72, "condition": "Unknown", "humidity": 50})
    print(f"  📊 {data['temp']}°F, {data['condition']}, {data['humidity']}% humidity")
    return data

In [ ]:
async def weather_agent():
    """Weather advisor with structured output"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o", temperature=0)
    session.services.register(model)

    # Create tool using Tool(func_callable=)
    weather_tool = Tool(func_callable=get_weather)

    # Note: Tools are automatically registered by react() - no need to register manually

    branch = session.create_branch(name="weather")

    print("🤔 Question: What's the weather in Seattle and what should I wear?\n")

    result = await react(
        session=session,
        branch=branch,
        parameters={
            "instruction": "What's the weather in Seattle and what should I wear?",
            "imodel": model.name,
            "model_name": model.name,  # Required for tool schema generation
            "tools": [weather_tool],  # Pass Tool instances
            "response_model": WeatherReport,  # Request structured output
            "max_steps": 3,
        },
    )

    # Parse structured response
    weather: WeatherReport = result.final_response

    print("\n🌦️  Weather Report:")
    print("=" * 60)
    print(f"📍 City: {weather.city}")
    print(f"🌡️  Temperature: {weather.temperature}°F")
    print(f"☁️  Condition: {weather.condition}")
    print(f"💧 Humidity: {weather.humidity}%")
    print(f"\n👕 Recommendation: {weather.recommendation}")
    print("=" * 60)

    return result


# Run weather agent
result = await weather_agent()

## Example 6: Parallel Tool Calling

Some models (like GPT-4) support calling multiple tools in parallel within a single step. This is useful for independent operations.

In [ ]:
# Define stock market tools
def get_stock_price(symbol: str) -> float:
    """Get current stock price for a symbol"""
    prices = {
        "AAPL": 178.50,
        "GOOGL": 142.30,
        "MSFT": 380.20,
        "AMZN": 155.60,
        "TSLA": 248.80,
    }
    price = prices.get(symbol.upper(), 0.0)
    print(f"  💰 {symbol}: ${price}")
    return price


def get_company_info(symbol: str) -> dict:
    """Get company information for a stock symbol"""
    info = {
        "AAPL": {"name": "Apple Inc.", "sector": "Technology", "founded": 1976},
        "GOOGL": {"name": "Alphabet Inc.", "sector": "Technology", "founded": 1998},
        "MSFT": {"name": "Microsoft Corp.", "sector": "Technology", "founded": 1975},
        "AMZN": {"name": "Amazon.com Inc.", "sector": "E-Commerce", "founded": 1994},
        "TSLA": {"name": "Tesla Inc.", "sector": "Automotive", "founded": 2003},
    }
    company = info.get(symbol.upper(), {})
    print(f"  🏢 {symbol}: {company.get('name', 'Unknown')}")
    return company


def calculate_market_cap(price: float, shares_outstanding: int) -> float:
    """Calculate market capitalization"""
    market_cap = price * shares_outstanding
    print(f"  📊 Market cap: ${market_cap:,.0f}")
    return market_cap

In [ ]:
async def stock_analyzer():
    """Stock analysis with parallel tool calls"""
    session = Session()
    model = iModel(
        provider="openai",
        model="gpt-4o",  # GPT-4 supports parallel tool calling
        temperature=0,
    )
    session.services.register(model)

    # Create tools using Tool(func_callable=)
    price_tool = Tool(func_callable=get_stock_price)
    info_tool = Tool(func_callable=get_company_info)
    cap_tool = Tool(func_callable=calculate_market_cap)

    # Note: Tools are automatically registered by react() - no need to register manually

    branch = session.create_branch(name="stocks")

    print("🤔 Question: Compare Apple (AAPL) and Microsoft (MSFT) stocks\n")

    result = await react(
        session=session,
        branch=branch,
        parameters={
            "instruction": (
                "Compare Apple (AAPL) and Microsoft (MSFT) stocks. "
                "Give me prices and company information for both."
            ),
            "imodel": model.name,
            "model_name": model.name,  # Required for tool schema generation
            "tools": [price_tool, info_tool, cap_tool],  # Pass Tool instances
            "max_steps": 3,
        },
    )

    print(f"\n📈 Stock Analysis:\n{result.final_response}")

    # Check if tools were called in parallel
    print("\n🔍 Checking for parallel tool execution:")
    for step in result.steps:
        if step.actions_requested and len(step.actions_requested) > 1:
            print(f"  ✅ Step {step.step}: {len(step.actions_requested)} tools called in parallel!")
            for action in step.actions_requested:
                print(f"     - {action.function}({action.arguments})")

    return result


# Run stock analyzer
result = await stock_analyzer()

## Key Takeaways

### 1. Tool Definition
```python
# From function using Tool(func_callable=)
tool = Tool(func_callable=my_function)

# Type hints are critical for schema generation
def my_tool(x: float, y: float) -> float:
    """Clear docstring helps LLM understand when to use this tool"""
    return x + y
```

### 2. ReAct Loop Control
```python
result = await react(
    session=session,
    branch=branch,
    parameters={
        "max_steps": 5,           # Limit iterations (prevent infinite loops)
        "response_model": Model,  # Optional structured output
    }
)
```

### 3. Error Handling
- Tools can raise exceptions
- LLM observes the error message
- Can retry with corrected arguments
- Set reasonable `max_steps` to prevent infinite retries

### 4. Best Practices
- ✅ Use clear function names and docstrings
- ✅ Add type hints for all parameters
- ✅ Register tools before calling `react()`
- ✅ Set `max_steps` to prevent infinite loops
- ✅ Make tools focused (single responsibility)
- ⚠️ Avoid stateful tools (use session state instead)
- ⚠️ Keep tool execution fast (<5 seconds)

<cell_type>markdown</cell_type>## Common Pitfalls

### ❌ Passing tool names instead of Tool instances
```python
tool = Tool(func_callable=my_func)
# WRONG: tools expects Tool instances, not strings
result = await react(parameters={"tools": [tool.name]})

# CORRECT: Pass Tool instances
result = await react(parameters={"tools": [tool]})
```

### ❌ Missing model_name parameter
```python
# WRONG: model_name is required for tool schema generation
result = await react(parameters={"imodel": model.name, "tools": [tool]})

# CORRECT: Include model_name
result = await react(parameters={
    "imodel": model.name,
    "model_name": model.name,  # Required!
    "tools": [tool],
})
```

### ❌ Missing type hints
```python
def bad_tool(x, y):  # No types = poor schema
    return x + y

# CORRECT: Always use type hints
def good_tool(x: float, y: float) -> float:
    """Add two numbers together."""
    return x + y
```

### ❌ No max_steps (infinite loop risk)
```python
result = await react(...)  # Could loop forever!

# CORRECT: Always set max_steps
result = await react(parameters={"max_steps": 10})
```

### ✅ Complete correct pattern
```python
# Create tools
tool1 = Tool(func_callable=my_func1)
tool2 = Tool(func_callable=my_func2)

# Run react with correct parameters
result = await react(
    session=session,
    branch=branch,
    parameters={
        "instruction": "Do something",
        "imodel": model.name,
        "model_name": model.name,
        "tools": [tool1, tool2],  # Tool instances, not names
        "max_steps": 10,
    }
)
```

## Next Steps

Now that you understand tool calling, explore:

1. **Multi-Agent Workflows** - Combine multiple specialized agents
2. **Streaming** - Stream tool calls and results in real-time
3. **Custom Tools** - Build domain-specific tools for your use case
4. **Tool Chaining** - Create complex workflows with dependent tools

**Resources**:
- [lionpride docs](https://github.com/khive-ai/lionpride)
- [Tool API reference](../docs/api/services.md)
- [ReAct pattern deep dive](../docs/patterns/react.md)

---

**You've completed the Tool Calling notebook! 🎉**

You now know how to:
- Define tools from Python functions
- Use `react()` for multi-step reasoning
- Handle errors and structured outputs
- Build practical agentic workflows